<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #FF1493; background-color: #ffffff;">Bristol-Myers Squibb – Molecular Translation</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">Image Captioning Tutorial With TPU Basics - Inference</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

<br>

---

**Learn how to use Tensorflow, TFRecords and TPUs to create an Image Captioning Pipeline That Will Yield The Following Performance - All Within 1 9HR Session**

- ~25 Levenshtein Distance On CV (EfficientNetB3 + 1024/512 LSTM) 
- ~TBD  Levenshtein Distance on Public LB (EfficientNetB3 + 1024/512 LSTM)
- ~5  Levenshtein Distance on CV (EfficientNetB7 + 1024/512 LSTM)
- ~TBD  Levenshtein Distance on Public LB (EfficientNetB7 + 1024/512 LSTM)

I really, really tried to make this notebook as understandable as possible. I hope it helps!

<br>

***NOTE - All of these models still had room for improvement. More finetuning could easily produce better scores***

<br>

---

<br>

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <br><center><b style="font-size: 16px;">🛑🛑🛑 &nbsp; CAUTION:  THIS NOTEBOOK IS A WORK IN PROGRESS  &nbsp; 🛑🛑🛑</b></center><br>
</div>

<br>


---

<br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🌒🌓🌔🌕🌖🌗🌘 &nbsp; MY OTHER NOTEBNOOKS</b><br><br><i>Here are the links to my other notebooks. Note that at this time the inference notebook is not open source. It will be soon.</i><br><br>
    - <a src="https://www.kaggle.com/dschettler8845/bms-cnn-attn-lstm-tpu-tutorial-inference"><b>Inference [this] Notebook</b></a><br>
    - <a src="https://www.kaggle.com/dschettler8845/bms-cnn-attn-lstm-tpu-tutorial-training"><b>Training Notebook</b></a><br>
    - <a src="https://www.kaggle.com/dschettler8845/bms-simple-tfrecord-creation"><b>TF Record Generation [384x384]</b></a><br>
    - <a src="https://www.kaggle.com/dschettler8845/bms-simple-tfrecord-creation-256x256"><b>TF Record Generation [256x256]</b></a><br>
    - <a src="https://www.kaggle.com/dschettler8845/bms-simple-tfrecord-creation-128x128"><b>TF Record Generation [128x128]</b></a><br>
</div>

<br>

---

<br>

<div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🙏 &nbsp; CREDIT TO THE FOLLOWING NOTEBOOKS I USED IN CREATING THIS KERNEL:</b><br><br><i>If you liked this notebook please upvote these other notebooks. Without them I wouldn't have been able to make this!</i><br><br>- <a src="https://www.kaggle.com/yihdarshieh/detailed-guide-to-custom-training-with-tpus"><b>Awesome Notebook For Best Practices in Distributed Computing</b></a><br>- <a src="https://www.kaggle.com/markwijkhuizen/tensorflow-tpu-training-baseline-lb-16-92/comments"><b>The Amazing Mark Wijkhuizen's TPU Training Notebook For This Competition</b></a><br>
</div>

<br>

<p id="toc"></p>

<br><br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#dataset_preparation">4&nbsp;&nbsp;&nbsp;&nbsp;PREPARE THE DATASET</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#model_preparation">5&nbsp;&nbsp;&nbsp;&nbsp;MODEL PREPARATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#model_training">6&nbsp;&nbsp;&nbsp;&nbsp;MODEL TRAINING</a></h3>

---

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;" id="imports">0&nbsp;&nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In [ ]:
# Installs
# print("\n... PIP INSTALLS STARTING ...\n")
# !pip install -q --upgrade pip
# !pip install -q pydot
# !pip install -q pydotplus
# !apt-get -qq install graphviz
# !pip install -q git+https://github.com/qubvel/classification_models.git
# !pip install -q git+https://github.com/qubvel/efficientnet.git
# print("\n... PIP INSTALLS COMPLETE ...\n")


print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");

# Library used to easily calculate LD
import Levenshtein

# Imports for Alternative Models
# import classification_models
# from classification_models.keras import Classifiers
# import efficientnet.keras as efn

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

print("\n\n... IMPORTS COMPLETE ...\n")
    
print("\n... SEEDING FOR DETERMINISTIC BEHAVIOUR ...\n")
seed_it_all()

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

<br><b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">PRIMARY TASK DESCRIPTION</b>


**Given an image, our goal is to generate a caption. In this case, that image is of a single molecule and the description/caption is the InChI string for that molecule.**

---

<br>

<b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">SECONDARY TASK DESCRIPTION</b>

In this notebook, we will go through, step by step, training models with TPUs in a custom way. The following steps will be covered:
* Use **`tf.data.Dataset`** as input pipeline
* Perform a custom training loop
* Correctly define loss function
* Gradient accumulation with TPUs<br>

<br>

<b style="text-decoration: underline; font-family: Verdana; text-transform: uppercase;">MORE DETAIL ON IMAGE CAPTIONING</b>


<b><sub><a href="https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/">Description From a Tutorial I Used As Reference</a></sub></b>

>Caption generation is a challenging artificial intelligence problem where a textual description must be generated for a given photograph.
>
>It requires both methods from computer vision to understand the content of the image and a language model from the field of natural language processing to turn the understanding of the image into words in the right order. Recently, deep learning methods have achieved state-of-the-art results on examples of this problem.
>
>Deep learning methods have demonstrated state-of-the-art results on caption generation problems. What is most impressive about these methods is a single end-to-end model can be defined to predict a caption, given a photo, instead of requiring sophisticated data preparation or a pipeline of specifically designed models.


<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;"  id="setup">2&nbsp;&nbsp;SETUP&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.1 ACCELERATOR DETECTION</h3>

---

In order to use **`TPU`**, we use **`TPUClusterResolver`** for the initialization which is necessary to connect to the remote cluster and initialize cloud TPUs. Let's go over two important points

1. When using TPU on Kaggle, you don't need to specify arguments for **`TPUClusterResolver`**
2. However, on **G**oogle **C**ompute **E**ngine (**GCE**), you will need to do the following:

<br>

```python
# The name you gave to the TPU to use
TPU_WORKER = 'my-tpu-name'

# or you can also specify the grpc path directly
# TPU_WORKER = 'grpc://xxx.xxx.xxx.xxx:8470'

# The zone you chose when you created the TPU to use on GCP.
ZONE = 'us-east1-b'

# The name of the GCP project where you created the TPU to use on GCP.
PROJECT = 'my-tpu-project'

tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER, zone=ZONE, project=PROJECT)
```

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- Although the Tensorflow documentation says it is the <b>project name</b> that should be provided for the argument <b><code>`project`</code></b>, it is actually the <b>Project ID</b>, that you should provide. This can be found on the GCP project dashboard page.<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCES:</b><br><br>
    - <a href="https://www.tensorflow.org/guide/tpu#tpu_initialization"><b>Guide - Use TPUs</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/distribute/cluster_resolver/TPUClusterResolver"><b>Doc - TPUClusterResolver</b></a><br>

</div>

In [ ]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy() 

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.2 COMPETITION DATA ACCESS</h3>

---

TPUs read data must be read directly from **G**oogle **C**loud **S**torage **(GCS)**. Kaggle provides a utility library – **`KaggleDatasets`** – which has a utility function **`.get_gcs_path`** that will allow us to access the location of our input datasets within **GCS**.<br><br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; TIPS:</b><br><br>- If you have multiple datasets attached to the notebook, you should pass the name of a specific dataset to the <b><code>`get_gcs_path()`</code></b> function. <i>In our case, the name of the dataset is the name of the directory the dataset is mounted within.</i><br><br>
</div>

In [ ]:
print("\n... DATA ACCESS SETUP STARTED ...\n")

if TPU:
    # Google Cloud Dataset path to training and validation images
    DATA_DIR = KaggleDatasets().get_gcs_path('bms-test-dataset-192x384')
else:
    # Local path to training and validation images
    DATA_DIR = "/kaggle/input/bms-test-dataset-192x384"
    
print(f"\n... DATA DIRECTORY PATH IS:\n\t--> {DATA_DIR}")
print(f"\n... IMMEDIATE CONTENTS OF DATA DIRECTORY IS:")
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

print("\n\n... DATA ACCESS SETUP COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.3 LEVERAGING MIXED PRECISION</h3>

---

Mixed precision is the use of both **`16-bit`** and **`32-bit`** floating-point types in a model during training to make it run faster and use less memory. By keeping certain parts of the model in the **`32-bit`** types for numeric stability, the model will have a lower step time and train equally as well in terms of the evaluation metrics such as accuracy. 

Today, most models use the **`float32`** dtype, which takes **`32`** bits of memory. However, there are two lower-precision dtypes, **`float16`** and **`bfloat16`**, each which take **`16`** bits of memory instead. Modern accelerators can run operations faster in the **`16-bit`** dtypes, as they have specialized hardware to run **`16-bit`** computations and **`16-bit`** dtypes can be read from memory faster.<br><br>

**NVIDIA GPUs** can run operations in **`float16`** faster than in **`float32`**<br>
**TPUs** can run operations **`bfloat16`** faster than in **`float32`**<br><br>

Therefore, these lower-precision dtypes should be used whenever possible on those devices. However, variables and a few computations should still be in **`float32`** for numeric reasons so that the model trains to the same quality. 

The Keras mixed precision API allows you to use a mix of either **`float16`** or **`bfloat16`** with **`float32`**, to get the performance benefits from **`float16/bfloat16`** and the numeric stability benefits from **`float32`**.<br><br>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; DEFINITION:</b><br><br>- The term <b>"numeric stability"</b> refers to how a model's quality is affected by the use of a lower-precision dtype instead of a higher precision dtype. We say an operation is "numerically unstable" in float16 or bfloat16 if running it in one of those dtypes causes the model to have worse evaluation accuracy or other metrics compared to running the operation in float32.<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>    - <a href="https://www.tensorflow.org/guide/mixed_precision"><b>TF Mixed Precision Overview</b></a><br>
</div>

In [ ]:
print(f"\n... MIXED PRECISION SETUP STARTING ...\n")
print("\n... SET TF TO OPERATE IN MIXED PRECISION – `bfloat16` – IF ON TPU ...")

# Set Mixed Precision Global Policy
#     ---> To use mixed precision in Keras, you need to create a `tf.keras.mixed_precision.Policy`
#          typically referred to as a dtype policy. 
#     ---> Dtype policies specify the dtypes layers will run in
tf.keras.mixed_precision.set_global_policy('float32')

# target data type, bfloat16 when using TPU to improve throughput
TARGET_DTYPE = tf.float32
print(f"\t--> THE TARGET DTYPE HAS BEEN SET TO {TARGET_DTYPE} ...")

# The policy specifies two important aspects of a layer: 
#     1. The dtype the layer's computations are done in
#     2. The dtype of a layer's variables. 
print(f"\n... TWO IMPORTANT ASPECTS OF THE GLOBAL MIXED PRECISION POLICY:")
print(f'\t--> COMPUTE DTYPE  : {tf.keras.mixed_precision.global_policy().compute_dtype}')
print(f'\t--> VARIABLE DTYPE : {tf.keras.mixed_precision.global_policy().variable_dtype}')

print(f"\n\n... MIXED PRECISION SETUP COMPLTED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.4 LEVERAGING XLA OPTIMIZATIONS</h3>

---


**XLA** (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes. **The results are improvements in speed and memory usage**.

<br>

When a TensorFlow program is run, all of the operations are executed individually by the TensorFlow executor. Each TensorFlow operation has a precompiled GPU/TPU kernel implementation that the executor dispatches to.

XLA provides us with an alternative mode of running models: it compiles the TensorFlow graph into a sequence of computation kernels generated specifically for the given model. Because these kernels are unique to the model, they can exploit model-specific information for optimization.<br><br>

<div class="alert alert-block alert-danger" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">🛑 &nbsp; WARNING:</b><br><br>- XLA can not currently compile functions where dimensions are not inferrable: that is, if it's not possible to infer the dimensions of all tensors without running the entire computation<br>
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- XLA compilation is only applied to code that is compiled into a graph (in <b>TF2</b> that's only a code inside <b><code>tf.function</code></b>).<br>- The <b><code>jit_compile</code></b> API has must-compile semantics, i.e. either the entire function is compiled with XLA, or an <b><code>errors.InvalidArgumentError</code></b> exception is thrown)
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>    - <a href="https://www.tensorflow.org/xla"><b>XLA: Optimizing Compiler for Machine Learning</b></a><br>
</div>

In [ ]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.5 BASIC DATA DEFINITIONS & INITIALIZATIONS</h3>

---


In [ ]:
# which_to_rotate = pd.read_csv("../input/bms-cnn-attn-lstm-tpu-tutorial-inference/sample_submission_w_extra_w_rotation.csv")
# pred_df = pd.read_csv("../input/bms-cnn-attn-lstm-tpu-tutorial-inference/prediction_confidences_for_rotation_analysis.csv")
# ss_df = pd.read_csv("../input/bms-molecular-translation/sample_submission.csv")

# pred_df["needs_rotation"] = which_to_rotate["needs_rotation"]
# pred_df["best_inchi"] = pred_df.progress_apply(lambda row: (row["w_rotated_inchi"] if row["needs_rotation"] else row["non_rotated_inchi"]), axis=1)
# ss_df.InChI=pred_df.best_inchi
# ss_df.to_csv("submission.csv", index=False)

In [ ]:
print("\n... BASIC DATA SETUP STARTING ...\n")

# All the possible tokens in our InChI 'language'
TOKEN_LIST = ["<PAD>", "InChI=1S/", "<END>", "/c", "/h", "/m", "/t", "/b", "/s", "/i"] +\
             ['Si', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'C', 'H', 'B', ] +\
             [str(i) for i in range(167,-1,-1)] +\
             ["\+", "\(", "\)", "\-", ",", "D", "T"]
print(f"\n... TOKEN LIST:")
for i, tok in enumerate(TOKEN_LIST): print(f"\t--> INTEGER-IDX = {i:<3}  –––  STRING = {tok}")

# The start/end/pad tokens will be removed from the string when computing the Levenshtein distance
# We want them as tf.constant's so they will operate properly within the @tf.function context
START_TOKEN = tf.constant(TOKEN_LIST.index("InChI=1S/"), dtype=tf.uint8)
END_TOKEN = tf.constant(TOKEN_LIST.index("<END>"), dtype=tf.uint8)
PAD_TOKEN = tf.constant(TOKEN_LIST.index("<PAD>"), dtype=tf.uint8)

# Prefixes and Their Respective Ordering/Format
#      -- ORDERING --> {c}{h/None}{b/None}{t/None}{m/None}{s/None}{i/None}{h/None}{t/None}{m/None}
PREFIX_ORDERING = "chbtmsihtm"
print(f"\n... PREFIX ORDERING IS {PREFIX_ORDERING} ...")

# Paths to Respective Image Directories
TEST_DIR = os.path.join(DATA_DIR, "test_records")

# Get the Full Paths to The Individual TFRecord Files
TEST_TFREC_PATHS = sorted(
    tf.io.gfile.glob(os.path.join(TEST_DIR, "*.tfrec")), 
    key=lambda x: int(x.rsplit("_", 2)[1]))

print(f"\n... TFRECORD INFORMATION:")
for SPLIT, TFREC_PATHS in zip(["TEST",], [TEST_TFREC_PATHS,]):
    print(f"\t--> {len(TFREC_PATHS):<3} {SPLIT:<5} TFRECORDS")

# Paths to relevant CSV files containing training and submission information
# SS_CSV_PATH    = os.path.join("/kaggle/input", "bms-csv-files-with-extra-metadata", "sample_submission_w_extra.csv")
SS_CSV_PATH = "../input/bms-molecular-translation/sample_submission.csv"
print(f"\n... PATHS TO CSVS:")
print(f"\t--> SS CSV   : {SS_CSV_PATH}")

# When debug is true we use a smaller batch size and smaller model
DEBUG=False

print("\n\n... BASIC DATA SETUP COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.6 INITIAL DATAFRAME INSTANTIATION</h3>

---


In [ ]:
print("\n... INITIAL DATAFRAME INSTANTIATION STARTING ...\n")

# Load the submission dataframe
ss_df    = pd.read_csv(SS_CSV_PATH)

# --- Distribution Information ---
N_TEST  = len(ss_df)

# --- Batching Information ---
BATCH_SIZE_DEBUG   = 2
REPLICA_BATCH_SIZE = 64

if DEBUG:
    REPLICA_BATCH_SIZE = BATCH_SIZE_DEBUG
OVERALL_BATCH_SIZE = REPLICA_BATCH_SIZE*N_REPLICAS

# --- Required Amount of Padding To Ensure Fixed Batch Sizes (no dropping) ---
REQUIRED_DATASET_PAD = OVERALL_BATCH_SIZE-N_TEST%OVERALL_BATCH_SIZE

# --- Input Image Information ---
IMG_SHAPE = (192,384,3)

# --- Autocalculate Training/Validation Information ---
TEST_STEPS = int(np.ceil(N_TEST/OVERALL_BATCH_SIZE))

# --- Modelling Information ---
ATTN_EMB_DIM  = 256
N_RNN_UNITS   = 768

print(f"\n... # OF TEST EXAMPLES       : {N_TEST:<7} ...\n")

print(f"\n... REPLICA BATCH SIZE    : {REPLICA_BATCH_SIZE} ...")
print(f"... OVERALL BATCH SIZE    : {OVERALL_BATCH_SIZE} ...\n")

print(f"\n... AMOUNT OF PADDING TO MAKE FULL BATCHES : {REQUIRED_DATASET_PAD} ...\n")

print(f"\n... IMAGE SHAPE           : {IMG_SHAPE} ...\n")

print(f"\n... TEST STEPS PER EPOCH : {TEST_STEPS:<5} ...\n")

print(f"\n... ATTENTION EMBEDDING DIMENSION : {ATTN_EMB_DIM:<5} ...")
print(f"... NUMBER OF UNITS IN LSTM       : {N_RNN_UNITS:<5} ...\n")

print("\n... SUBMISSION DATAFRAME ...\n")
display(ss_df.head(3))

print("\n... INITIAL DATAFRAME INSTANTIATION COMPLETED...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">2.7 USER INPUT VARIABLES</h3>

---


In [ ]:
print("\n... SPECIAL VARIABLE SETUP STARTING ...\n")

ENCODER_CKPT_PATH = "/kaggle/input/new-bms-image-captioning-train-tpu/encoder_epoch_safety_save.h5"
DECODER_CKPT_PATH = "/kaggle/input/new-bms-image-captioning-train-tpu/decoder_epoch_safety_save.h5"

# Paths to model weights to use in inference
print(f"\n... ENCODER MODEL INFERENCE WILL USE CHECKPOINT:\n\t-->{ENCODER_CKPT_PATH}\n")
print(f"... DECODER MODEL INFERENCE WILL USE CHECKPOINT:\n\t-->{DECODER_CKPT_PATH}\n")
    
print("\n... SPECIAL VARIABLE SETUP COMPLETED ...\n")

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="helper_functions">
    3&nbsp;&nbsp;HELPER FUNCTION & CLASSESS&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a>
</h1>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">3.1 GENERAL HELPER FUNCTIONS</h3>

---

In [ ]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]


def tf_load_image(path, img_size=(192,384,3), invert=False):
    """ Load an image with the correct size and shape 
    
    Args:
        path (tf.string): Path to the image to be loaded
        img_size (tuple, optional): Size to reshape image to (required for TPU)
        tile_to_3_channel (bool, optional): Whether to tile the single channel
            image to 3 channels which will be required for most off-the-shelf models
        invert (bool, optional): Whether or not to invert the background/foreground
    
    Returns:
        3 channel tf.Constant image ready for training/inference
    
    """
    img = decode_img(tf.io.read_file(path), img_size, n_channels=3, invert=invert)        
    return img

    
def pad_to_square(a, constant=255):
    """ Pad a tensor array `a` evenly until it is a square 
    
    Args:
        a (tf.constant array): Array to be padded to become square (3D)
        constant (int, optional): Value to use in padding
    
    Returns:
        square constant padded array
    """
    h_src = tf.shape(a)[0]
    w_src = tf.shape(a)[1] 
    if w_src>h_src: # pad height
        n_to_add = w_src-h_src
        top_pad = n_to_add//2
        bottom_pad = n_to_add-top_pad
        a = tf.pad(a, [(top_pad, bottom_pad), (0, 0), (0, 0)], mode='constant', constant_values=constant)
    elif h_src>w_src: # pad width
        n_to_add = h_src-w_src
        left_pad = n_to_add//2
        right_pad = n_to_add-left_pad
        a = tf.pad(a, [(0, 0), (left_pad, right_pad), (0, 0)], mode='constant', constant_values=constant)
    else:
        pass
    return a
    
    
def decode_image(image_data, resize_to=(384,384,3)):
    """ Function to decode the tf.string containing image information 
    
    
    Args:
        image_data (tf.string): String containing encoded image data from tf.Example
        resize_to (tuple, optional): Size that we will reshape the tensor to (required for TPU)
    
    Returns:
        Tensor containing the resized single-channel image in the appropriate dtype
    """
    image = tf.image.decode_png(image_data, channels=3)
    image = tf.reshape(image, resize_to)
    return tf.cast(image, TARGET_DTYPE)


def tokens_to_str(caption_tokens, discard_padding=True):
    """ Should convert a string of token ids to an InChI string """
    if discard_padding:
        return "".join([int_2_char_lex[x] for x in caption_tokens if x!=len(int_2_char_lex)])
    else:
        return "".join([int_2_char_lex[x] for x in caption_tokens])
    
    
def evaluate(image, from_np=False):
    """ TBD """
    attention_plot = np.zeros((MAX_LEN, fixed_encoder.output_shape[1]))
    hidden = tf.zeros((1, RNN_UNITS), tf.float32)

    if not from_np:
        temp_input = tf.expand_dims(tf_load_image(image, img_size=INPUT_SHAPE[:-1]), 0)
        img_tensor_val = fixed_encoder(temp_input)
    else:
        img_tensor_val=image
    
    features = trainable_encoder(img_tensor_val)
    dec = tf.ones((1, 1), tf.uint8)
    result = [int_2_char_lex[1],]

    for i in range(MAX_LEN-1):
        predictions, hidden, attention_weights = dec_model([dec, hidden, features])
        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(int_2_char_lex[predicted_id])
        if int_2_char_lex[predicted_id] == '<END>':
            return result, attention_plot

        dec = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot


def plot_attention(image, result, attention_plot):
    """ TBD """    
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(18, 14))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.4, extent=img.get_extent())

    plt.tight_layout()
    plt.show()
    
def test_random_image(style="full"):
    """ TBD """    
    rid = np.random.randint(0, len(val_subset_df))
    path = val_subset_df["img_path"][rid]
    
    if style=="full":
        real_caption = val_subset_df["InChI"][rid][:-1]
    else:
        real_caption = val_subset_df["InChI_chem"][rid][:-1]

    result, attention_plot = evaluate(path)
    result = ''.join(result[:-1])
    print (f"\n\tReal Caption       : {real_caption}")
    print (f"\tPrediction Caption   : {result}")
    print(f"\tLevenshtein Distance  : {Levenshtein.distance(real_caption, result)}\n")
    plot_attention(path, result, attention_plot)
    
    
# sparse tensors are required to compute the Levenshtein distance
def dense_to_sparse(dense):
    """ Convert a dense tensor to a sparse tensor 
    
    Args:
        dense (Tensor): TBD
        
    Returns:
        A sparse tensor    
    """
    indices = tf.where(tf.ones_like(dense))
    values = tf.reshape(dense, (MAX_LEN*OVERALL_BATCH_SIZE,))
    sparse = tf.SparseTensor(indices, values, dense.shape)
    return sparse

def get_levenshtein_distance(preds, lbls):
    """ Computes the Levenshtein distance between the predictions and labels 
    
    Args:
        preds (tensor): Batch of predictions
        lbls (tensor): Batch of labels
        
    Returns:
        The mean Levenshtein distance calculated across the batch
    """
    preds = tf.where(tf.not_equal(lbls, END_TOKEN) & tf.not_equal(lbls, PAD_TOKEN), preds, 0)
    lbls = tf.where(tf.not_equal(lbls, END_TOKEN), lbls, 0)

    preds_sparse = dense_to_sparse(preds)
    lbls_sparse = dense_to_sparse(lbls)

    batch_distance = tf.edit_distance(preds_sparse, lbls_sparse, normalize=False)
    mean_distance = tf.math.reduce_mean(batch_distance)
    
    return mean_distance

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="dataset_preparation">4&nbsp;&nbsp;PREPARE THE DATASET&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In this section we prepare the **`tf.data.Datasets`** we will use for training and validation

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">4.0 CREATE THE TOKENIZED INCHI LABELS (TO BE REMOVED LATER)</h3>

---


This step is optional and should be replaced with the InChI labels directly encoded into the TFRecord files. That being said, we will do it like so for now to allow for easy modification of the tokenization styles.

In [ ]:
print("\n... CREATING THE TOKEN MAPPINGS AND IMPORTANT CONSTANTS ...\n")

print("\n... CREATE RELEVANT OBJECTS ... ")
tok_2_int = {c.strip("\\"):i for i,c in enumerate(TOKEN_LIST)}
int_2_tok = {v:k for k,v in tok_2_int.items()}

print("... CREATE IMPORTANT CONSTANTS ...\n")
# Max Length Was Determined Previously Using... 
#     >>> MAX_LEN = train_df.InChI.progress_apply(lambda x: len(re.findall("|".join(TOKEN_LIST), x))).max()+1
MAX_LEN = 277
VOCAB_LEN = len(int_2_tok)

print("\n... FINISHED CREATING THE TOKEN MAPPINGS AND IMPORTANT CONSTANTS ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">4.1 READ TFRECORD FILES - CREATE THE RAW DATASET(S)</h3>

---

Here we will leverage **`tf.data.TFRecordDataset`** to read the TFRecord files.
* The simplest way is to specify a list of filenames (paths) of TFRecord files.
* It is a subclass of **`tf.data.Dataset`**.

This newly created raw dataset contains **`tf.train.Example`** messages, and when iterated over it, we get scalar string tensors.

In [ ]:
print("\n... CREATE TFRECORD RAW DATASETS STARTING ...\n")

# Create tf.data.Dataset from filepaths for conversion later
raw_test_ds = tf.data.TFRecordDataset(TEST_TFREC_PATHS, num_parallel_reads=None)

print(f"\n... THE RAW TF.DATA.TFRECORDDATASET OBJECT:\n\t--> {raw_test_ds}\n")

print("\n... CREATE TFRECORD RAW DATASETS COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">4.2 PARSE THE RAW DATASET(S)</h3>

---


The general recipe to parse the string tensors in the raw dataset looks something like this:

<br>

**STEP 1.**  Create a description of the features. For example:

```python
feature_description = {    
    'feature0': tf.io.FixedLenFeature([], tf.int64),
    'feature1': tf.io.FixedLenFeature([], tf.string),
    'feature2': tf.io.FixedLenFeature([], tf.float32),
    ...
}
```

<br>

**STEP 2.**  Define a parsing function by using `tf.io.parse_single_example` and the defined feature description.
```python
def _parse_function(example):
    """
    Args:
        example: A string tensor representing a `tf.train.Example`.
    """

    # Parse `example`.
    parsed_example = tf.io.parse_single_example(example, feature_description)

    return parsed_example
```

<br>

**STEP 3.**  Map the raw dataset by `_parse_function`.
```python
dataset = raw_dataset.map(_parse_function)
```

<br>

---

<br>

**In the following cell, we apply the above recipe to our BMS tfrecord dataset.**


<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- The parsed images are <code><b>`tf.string`</b></code>, which are then decoded with <code><b>`tf.image.decode_png`</b></code> which is an alias for <code><b>`tf.io.decode_png`</b></code><br>- The InChI strings and Image IDs will just be left as byte string tensors.
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>
    - <a href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><b>Tutorial - TFRecord and tf.Example</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset"><b>TFRecordDataset Documentation</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/io/decode_png"><b>Decoding PNGs Documentation</b></a><br>
</div>


In [ ]:
def decode(serialized_example, is_test=False, tokenized_inchi=True):
    """ Parses a set of features and label from the given `serialized_example`.
        
        It is used as a map function for `dataset.map`

    Args:
        serialized_example (tf.Example): A serialized example containing the
            following features:
                – 'image'
                – 'image_id'
                – 'inchi'
        is_test (bool, optional): Whether to allow for the InChI feature
        drop_id (bool, optional): Whether or not to drop the ID feature
        
    Returns:
        A decoded tf.data.Dataset object representing the tfrecord dataset
    """
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
    }
    
    if not is_test:
        if tokenized_inchi:
            feature_dict["inchi"] = tf.io.FixedLenFeature(shape=[MAX_LEN], dtype=tf.int64, default_value=[0]*MAX_LEN)
        else:
            feature_dict["inchi"] = tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value='')
    else:
        feature_dict["image_id"] = tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value='')
    
    # Define a parser
    features = tf.io.parse_single_example(serialized_example, features=feature_dict)
    
    # Decode the tf.string
    image = decode_image(features['image'], resize_to=IMG_SHAPE)
    
    # Figure out the correct information to return
    if is_test:
        image_id = features["image_id"] 
        return image, image_id
    else:
        if tokenized_inchi:
            target = tf.cast(features["inchi"], tf.uint8)
        else:
            target = features["inchi"]
        return image, target

In [ ]:
print("\n... DECODING RAW TFRECORD DATASETS STARTING ...\n")

# Decode the tfrecords completely –– decode is our `_parse_function` (from recipe above)
test_ds = raw_test_ds.map(lambda x: decode(x, is_test=True))
print(f"\n... THE DECODED TF.DATA.TFRECORDDATASET OBJECT:" \
      f"\n\t--> ((image, image_id))" \
      f"\n\t--> {test_ds}\n")

print("\n... 2 EXAMPLES OF IMAGES AND LABELS AFTER DECODING ...")
for i, (img, image_id) in enumerate(test_ds.take(2)):
    print(f"\nIMAGE SHAPE : {img.shape}")
    print(f"\nIMAGE ID    : {image_id.numpy().decode()}")

    plt.figure(figsize=(10,10))
    plt.imshow(img.numpy().astype(np.int64), cmap="gray")
    plt.title(f"{image_id.numpy().decode()}")
    plt.show()

print("\n... DECODING RAW TFRECORD DATASETS COMPLETED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">4.4 WORKING WITH `TF.DATA.DATASET` OBJECTS</h3>

---

With the above parsing methods defined, we can define how to load the dataset with more options and further apply shuffling, bacthing, etc. In particular the following methods and attributes are of special interest to us:
* Use **`num_parallel_reads`** in **`tf.data.TFRecordDataset`** to read files in parallel.
* Set **`tf.data.Options.experimental_deterministic=False`** and use it to get a new dataset that ignores the order of elements.
* Use **`num_parallel_calls`** in **`tf.data.Dataset.map()`** method to have parallel processing.
* Use **`tf.data.Dataset.prefetch()`** to allow later batches to be prepared while the current batch is being processed.
* Use **`tf.data.AUTOTUNE`** to automatically determine parallelization argument values

The parallel processing and prefetching are particular important when working with TPU:
* This is because a TPU can process batches very quickly
* The dataset pipeline should be able to provide data for TPU efficiently, otherwise the TPU will be idle.

**In the cell below we will create the functions and configuration template which will later be used to create our respective datasets**

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>
    - <a href="https://www.tensorflow.org/guide/data"><b>Guide - tf.data: Build TensorFlow Input Pipelines</b></a><br>
    - <a href="https://www.tensorflow.org/guide/data_performance"><b>Guide - Better Performance With the tf.data API</b></a><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/data/Dataset"><b>tf.data.Dataset Documentation</b></a><br>
</div>

In [ ]:
def load_dataset(filenames, is_test=False, ordered=False, tokenized_inchi=True):
    """Read from TFRecords.
    
    For optimal performance, reading from multiple files at once and disregarding data order (if `ordered=False`).
        - If pulling InChI from TFRecords than order does not matter since we will 
          be shuffling the data anyway (for training dataset).
          
    Args:
        filenames (list of strings): List of paths to that point to the respective TFRecord files
        is_test (bool, optional): Whether or not to include the image ID or label in the returned dataset
        ordered (bool, optional): Whether to ensured ordered results or maximize parallelization
        tokenized_inchi (bool, optional): Whether our dataset includes the tokenized inchi or we will be 
            creating it from the caption numpy array
        
    Returns:
        Decoded tf.data.Dataset object
    """

    options = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        options.experimental_deterministic = False
        N_PARALLEL=tf.data.AUTOTUNE
    else:
        N_PARALLEL=None
        
    # If not-ordered, this will read in by automatically interleaving multiple tfrecord files.
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=N_PARALLEL)
    
    # If not-ordered, this will ensure that we use data as soon as it 
    # streams in, rather than in its original order.
    dataset = dataset.with_options(options) 
    
    # parse and return a dataset w/ the appropriate configuration
    dataset = dataset.map(
        lambda x: decode(x, is_test, tokenized_inchi),
        num_parallel_calls=N_PARALLEL,
    )
    
    return dataset

def get_dataset(filenames, batch_size, 
                is_test=False, 
                shuffle_buffer_size=1, 
                repeat_dataset=True, 
                preserve_file_order=False, 
                drop_remainder=True,
                tokenized_inchi=True,
                external_inchi_dataset=None,
                test_padding=0):
    """ Get a tf.data.Dataset w/ the appropriate configuration
    
    Args:
        TBD
        test_padding (int, optional): Amount required to pad dataset to have only full batches
        
    Returns:
        TBD
        
    """
    # Load the dataset
    dataset = load_dataset(filenames, is_test, preserve_file_order, tokenized_inchi)
    
    if test_padding!=0:
        pad_dataset = tf.data.Dataset.from_tensor_slices((
            tf.zeros((test_padding, *IMG_SHAPE), dtype=TARGET_DTYPE),       # Fake Images
            tf.constant(["000000000000",]*test_padding, dtype=tf.string))   # Fake IDs
        )
        dataset = dataset.concatenate(pad_dataset)
    
    # If we are training than we will want to repeat the dataset. 
    # We will determine the number of steps (or updates) later for 1 training epoch.
    if repeat_dataset:
        dataset = dataset.repeat()
    
    # If we need to add on manually the inchi
    if external_inchi_dataset is not None:
        # Zip the datasets and tile the 1 channel image to 3 channels & drop the old inchi value
        dataset = tf.data.Dataset.zip((dataset, external_inchi_dataset))
        dataset = dataset.map(lambda x,y: (tf.tile(tf.expand_dims(x[0], -1), tf.constant([1,1,3], tf.int32)), y))
                              
    # Shuffling
    if shuffle_buffer_size!=1:
        dataset = dataset.shuffle(shuffle_buffer_size)
    
    # Batching
    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [ ]:
####### ####### ####### ####### ####### ####### ####### #######

# Template Configuration
DS_TEMPLATE_CONFIG = dict(
    filenames=[],
    batch_size=1,
    is_test=False, 
    shuffle_buffer_size=1, 
    repeat_dataset=True, 
    preserve_file_order=False, 
    drop_remainder=True,
    tokenized_inchi=True,
    external_inchi_dataset=None,
    test_padding=0
)

####### ####### ####### ####### ####### ####### ####### #######

TEST_DS_CONFIG = DS_TEMPLATE_CONFIG.copy()
TEST_DS_CONFIG.update(dict(
    filenames=TEST_TFREC_PATHS,
    batch_size=OVERALL_BATCH_SIZE,
    is_test=True,
    repeat_dataset=False,
    drop_remainder=True,
    test_padding=REQUIRED_DATASET_PAD,
))

####### ####### ####### ####### ####### ####### ####### #######

test_ds = get_dataset(**TEST_DS_CONFIG)

for SPLIT, CONFIG in zip(["TESTING",], [TEST_DS_CONFIG]):
    print(f"\n... {SPLIT} CONFIGURATION:")
    for k,v in CONFIG.items():
        if k=="filenames":
            print(f"\t--> {k:<23}: {[path.split('/', 4)[-1] for path in v[:2]]+['...']}")
        else:
            print(f"\t--> {k:<23}: {v}")

print(f"... TESTING DATASET    : {test_ds}    ...\n")

print("\n\n ... SOME EXAMPLES ... \n\n")
for x,y in test_ds.take(1):
    for i in range(REPLICA_BATCH_SIZE-3, REPLICA_BATCH_SIZE, 1):
        plt.figure(figsize=(12,12))
        plt.imshow(x[i].numpy().astype(np.int64))
        plt.title(f"{y[i].numpy().decode()}")
        plt.show()

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="model_preperation">5&nbsp;&nbsp;MODEL PREPERATION&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In this section we prepare the models for training. We will be using a model architecture very similar to that found within the [**Show, Attend, and Tell Research Paper**](https://arxiv.org/pdf/1502.03044.pdf).

<br>

<center><img src="https://kelvinxu.github.io/projects/diags/model_diag.png" width=50%></center>
    
<br>

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.1 UNDERSTANDING THE MODELS - ENCODER</h3>

---

We will be leveraging an [**EfficientNet**](https://arxiv.org/pdf/1905.11946) model to act as the Encoder CNN in our network. Ideally I will update this to be an [**NFNet**](https://arxiv.org/pdf/2102.06171) or an [**EfficientNetV2**](https://arxiv.org/pdf/2104.00298) model in the near future... or perhaps [**ViT**](https://arxiv.org/pdf/2010.11929)??
* On **TPU** we will use an **EfficientNetB3** model *(In my private training notebook I leverage EB7)*
* On **GPU/CPU** we will use an **EfficientNetB0** model

<br>

<sub><sup>***Basic View of EfficientNetB0 Architecture w/ 380x380x3 Input***</sup></sub>
<center><img src="https://www.researchgate.net/publication/339462624/figure/fig1/AS:862263699316737@1582591094412/The-architecture-of-EfficientNet-b0.ppm" width=75%></center>

<br>

Our encoder will create feature maps for each image which will in turn be passed to the decoder side of the network. EfficientNetB0 creates $1280$ feature maps with dimensions of $12\cdot12$ pixels. These feature maps will be flattened by a reshape: $12\cdot12 \Rightarrow 144$. 

<br>

**In the following cell, we will create a function to generate our encoder model.**


<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- For different encoder architectures we will have a different number of feature maps. i.e. If we utilized <b>EfficientNetB3</b> we would have <b>1536</b> feature maps instead of the <b>1280</b> feature maps that <b>EfficientNetB0</b> produces. 
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>
    - <a href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><b>TF Tutorial – Transformer Model for Language Understanding</b></a><br>
    - <a href="https://www.tensorflow.org/tutorials/text/image_captioning"><b>TF Tutorial – Image Captioning</b></a><br>
    - <a href="https://www.tensorflow.org/tutorials/text/nmt_with_attention"><b>TF Tutorial – Neural Machine Translation w/ Attention</b></a><br>
</div>

In [ ]:
print("\n... ENCODER MODEL CREATION STARTING ...\n")

# SAMPLE IMAGES
SAMPLE_IMGS, SAMPLE_IDS = next(iter(test_ds.unbatch().batch(BATCH_SIZE_DEBUG)))

# ENCODER_CONFIG
USE_QUBVEL = False
PRETRAINED_WTS = None # or `noisy-student` is possible if using qubvel EfficientNet or `None`
if TPU and not DEBUG:
    if USE_QUBVEL:
        BB_FN = efn.EfficientNetB5
    else:
        BB_FN = tf.keras.applications.EfficientNetB5
else:
    if USE_QUBVEL:
        BB_FN = efn.EfficientNetB5
    else:
        BB_FN = tf.keras.applications.EfficientNetB5
PREPROCESSING_FN = tf.keras.applications.efficientnet.preprocess_input
    
# This will be the dimension the network outputs flattened
IMG_EMB_DIM = BB_FN(include_top=False, input_shape=IMG_SHAPE).output_shape[1:]
IMG_EMB_DIM = (IMG_EMB_DIM[0]*IMG_EMB_DIM[1], IMG_EMB_DIM[2])

class Encoder(tf.keras.Model):
    def __init__(self, image_embedding_dim, preprocessing_fn, backbone_fn, image_shape, do_permute=False, pretrained_weights="imagenet"):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        super(Encoder, self).__init__()
        
        self.image_embedding_dim = image_embedding_dim
        self.preprocessing_fn = preprocessing_fn
        self.encoder_backbone = backbone_fn(include_top=False, weights=None, input_shape=image_shape)        
        self.reshape = tf.keras.layers.Reshape(self.image_embedding_dim, name='image_embedding')
        
    def call(self, x, training):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        x = self.preprocessing_fn(x)
        x = self.encoder_backbone(x, training=training)
        x = self.reshape(x, training=training)
        return x
    
    
# Example enoder output
with tf.device('/CPU:0'):
    encoder = Encoder(IMG_EMB_DIM, PREPROCESSING_FN, BB_FN, IMG_SHAPE, do_permute=IMG_EMB_DIM[1]<IMG_EMB_DIM[0])
    img_embedding_batch = encoder(SAMPLE_IMGS)
print(f'\n... Encoder Output Shape  :  (batch_size, embedding_length, embedding_depth)  :  {img_embedding_batch.shape} ...\n')

print("\n... ENCODER MODEL CREATION FINISHED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.2 UNDERSTANDING THE MODELS - DECODER</h3>

---

We will be leveraging a decoder nearly identical to that found in the [**Show, Attend, and Tell Research Paper**](https://arxiv.org/pdf/1502.03044). This paper introduces an archiecture for image captioning that incorporates attention (Soft and Hard Attention). We will be utilizing Bahdanau Attention (Soft Attention) within this notebook. The core components of our decoder will be:

* **Attention Mechanism <sub></sup>[(Partly Quoting From Mark Wijkhuizen's Notebook)](https://www.kaggle.com/markwijkhuizen/tensorflow-tpu-training-baseline-lb-16-92)</sup></sub>**
    * The attention mechanism takes as input the hidden state from the LSTM, which is the LSTM state after the last predicted character, and encoder features. 
        * The hidden LSTM state will differ each prediction iteration, but the encoder result remains the same. 
        * Using this hidden LSTM state the attention mechanism learns which encoder features are important and which are not, given an LSTM state.
    * To make this idea of attention a bit less abstract, let us take the a random InChI string as an example...
        * **`C13H5F5N2/c14-7-3-6(5-19)1-2-10(7)20-13-11(17)8(15)4-9(16)12(13)18/h1-4,20H`**
        * Let's say the model has so far correctly predicted **`C13H5`**, the attention mechanism should now focus on features containing **`F`** atoms and disregard any feature maps that target **`C`** or **`H`** atoms. 
        * The LSTM hidden state should tell the attention mechanism it has predicted **`C13H5`** so far and the attention mechanism will learn it has to focus on **`F`** atoms after **`C`** and **`H`** atoms are predicted.
        * The idea of 'focus' in this case refers to giving weight to certain parts of the feature maps where the attention should believes should be important.<br><br>

* **Recurrent Mechanism – Embedding & LSTM/GRU <sub></sup>[(Partly Quoting From Mark Wijkhuizen's Notebook)](https://www.kaggle.com/markwijkhuizen/tensorflow-tpu-training-baseline-lb-16-92)</sup></sub>**
    * The Embedding layer will learn a representation to convert the chemical token/characters into a representative vector
    * The LSTMCell takes a concatenated context vector from the attention mechanism and an embedded token value as input. 
    * The LSTMCell hidden and carry states are initialized using the encoder features.

<br>

<sub><sup>***Basic Overview of the Image Captioning Process With Some Rudimentary Math***</sup></sub>
<center><img src="https://i.ibb.co/H2fkVpL/attn.gif" alt="attn_gif" border="0" width="80%"></center>

<br>

**In the following cell, we will create a function to generate our decoder model.**

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📌 &nbsp; NOTE:</b><br><br>- We decided to utilize an LSTM cell for the Recurrent portion of our decoder network. This decision is based on the concept that LSTMs can generally handle longer sequences better than GRUs. That being said, other options do exist. 
</div>

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>
    - <a href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><b>TF Tutorial – Transformer Model for Language Understanding</b></a><br>
    - <a href="https://www.tensorflow.org/tutorials/text/image_captioning"><b>TF Tutorial – Image Captioning</b></a><br>
    - <a href="https://www.tensorflow.org/tutorials/text/nmt_with_attention"><b>TF Tutorial – Neural Machine Translation w/ Attention</b></a><br>
</div>

In [ ]:
print("\n... ATTENTION MECHANISM LAYER CREATION STARTING ...\n")

# TO BE REPLACED W/ --> tfa.seq2seq.BahdanauAttention
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, attn_emb_depth):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(attn_emb_depth)
        self.W2 = tf.keras.layers.Dense(attn_emb_depth)
        self.V = tf.keras.layers.Dense(1)

    def call(self, hidden, features, training):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # features shape == (batch_size, img_emb_size, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, img_emb_size, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # img_emb_size refers to the product of the image embedding width and height (144 for EB)
        # the shape of the tensor before applying self.V is (batch_size, img_emb_size, attention_emb_depth)
        score = self.V(
            tf.nn.tanh(self.W1(hidden_with_time_axis, training=training) + \
                       self.W2(features, training=training)), 
            training=training
        )

        # attention_weights shape (before & after) == (batch_size, img_emb_size, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after multiplication is (batch_size, img_emb_size, 1)
        context_vector = attention_weights * features
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = tf.reduce_sum(context_vector, axis=1) # this is axis 2 in some implementations?
        
        return context_vector, attention_weights
    

# ATTN_EMB_DIM  = 512
# N_RNN_UNITS   = 1024
with tf.device('/CPU:0'):
    attn_layer = BahdanauAttention(ATTN_EMB_DIM)
    context_vector, attn_weights = attn_layer(tf.zeros([BATCH_SIZE_DEBUG, IMG_EMB_DIM[0]]), img_embedding_batch)

print(f'\n... Context Vector Shape     :  (batch_size, n_rnn_units)     :  {context_vector.shape}   ...')
print(f'... Attention Weights Shape  :  (batch_size, img_emb_dim, 1)  :  {attn_weights.shape} ...\n')

print("\n... ATTENTION MECHANISM LAYER CREATION FINISHED ...\n")

In [ ]:
print("\n... DECODER MODEL CREATION STARTING ...\n")

class Decoder(tf.keras.Model):
    def __init__(self, vocab_len, attn_emb_depth, img_emb_dim, n_rnn_units, dropout_rate=0.1):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        super(Decoder, self).__init__()
        
        # Basic Parameters
        self.vocab_len = vocab_len
        self.attn_emb_depth = attn_emb_depth
        self.img_emb_dim = img_emb_dim
        self.n_rnn_units = n_rnn_units
        self.dropout_rate = dropout_rate
        
        # Attention Mechanism
        self.attention_layer = BahdanauAttention(self.attn_emb_depth)
        
        # LSTM hidden and carry state initialization
        self.init_h_layer = tf.keras.layers.Dense(
            units=n_rnn_units, input_shape=[img_emb_dim], name='img_embedding__hidden_init_layer'
        )
        self.init_c_layer = tf.keras.layers.Dense(
            units=n_rnn_units, input_shape=[img_emb_dim], name='img_embedding__input_act_init_layer'
        )
        
        # Character Embedding Layer In The Decoder
        #    - TODO ... `mask_zero=True` ???
        self.embedding_layer = tf.keras.layers.Embedding(vocab_len, n_rnn_units, )
        
        # The LSTM cell
        self.lstm_cell_layer = tf.keras.layers.LSTMCell(n_rnn_units, name='lstm_cell_layer')
        
        # Dropout Layer to Prevent Overfitting
        self.dropout_layer = tf.keras.layers.Dropout(self.dropout_rate, name='dropout_layer')
        
        # Fully Connected Prediction Layer 
        self.fcn_layer = tf.keras.layers.Dense(units=vocab_len, input_shape=[n_rnn_units], dtype=tf.float32, name='fc_prediction_layer')

    def call(self, token, hidden, memory, img_emb, training):
        """ TODO
        
        Args:
            TODO        
        
        Returns:
            TODO
        """
        # img_emb shape == (batch_size, img_emb_shape, hidden_size)
        #    --> We ignore the attention weights for now (_)
        context_vector, _ = self.attention_layer(hidden, img_emb, training=training)

        # shape after passing token through embedding == (batch_size, 1, n_rnn_units)
        # shape after passing through squeeze == (batch_size, n_rnn_units)
        tok_emb = tf.squeeze(self.embedding_layer(token, training=training), axis=1)
        
        # shape after concatenation == (batch_size, n_rnn_units + hidden_size)
        x = tf.concat((context_vector, tok_emb), axis=-1)

        # passing the concatenated vector to the LSTM cell
        #    - also getting the new hidden (h) and memory (c) vectors
        _, (hidden_new, memory_new) = self.lstm_cell_layer(x, (hidden, memory), training=training)
        
        #### ##### ##### ####
        # MORE LSTM LAYERS? #
        #### ##### ##### ####
        
       # compute prediction logits and leverage dropout
        output = self.dropout_layer(hidden_new, training=training)
        output = self.fcn_layer(output, training=training)

        return output, hidden_new, memory_new
    
    def init_hidden_state(self, img_emb, training):
        mean_encoder_out = tf.math.reduce_mean(img_emb, axis=1)
        hidden = self.init_h_layer(mean_encoder_out, training=training)  # (batch_size, n_rnn_units)
        memory = self.init_c_layer(mean_encoder_out, training=training)
        return hidden, memory


with tf.device('/CPU:0'):
    decoder = Decoder(VOCAB_LEN, ATTN_EMB_DIM, IMG_EMB_DIM[0], N_RNN_UNITS)
    h, c = decoder.init_hidden_state(img_embedding_batch[:BATCH_SIZE_DEBUG], training=False)
    pred_output, h, c = decoder(tf.random.uniform((BATCH_SIZE_DEBUG, 1)), h, c, img_embedding_batch)

print(f'\n... Decoder Output Shape  :  (batch_size, vocab_len)  :  {pred_output.shape} ...\n')

print("\n... DECODER MODEL CREATION FINISHED ...\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.4 WRAP THE CONFIGURATION DETAILS IN A CLASS OBJECT FOR EASY ACCESS</h3>

---


In [ ]:
class Config():
    def __init__(self,):
        self.encoder_config = {}
        self.decoder_config = {}
    def initialize_encoder_config(self, image_embedding_dim, preprocessing_fn, backbone_fn, image_shape, do_permute=False, pretrained_weights="imagenet"):
        self.encoder_config = dict(
            image_embedding_dim=image_embedding_dim, 
            preprocessing_fn=preprocessing_fn, 
            backbone_fn=backbone_fn, 
            image_shape=image_shape, 
            do_permute=do_permute, 
            pretrained_weights=pretrained_weights,
        )
    def initialize_decoder_config(self, vocab_len, attn_emb_depth, img_emb_dim, n_rnn_units, dropout_rate=0.1):
        self.decoder_config = dict(
            vocab_len=vocab_len, 
            attn_emb_depth=attn_emb_depth, 
            img_emb_dim=img_emb_dim, 
            n_rnn_units=n_rnn_units, 
            dropout_rate=dropout_rate,
        )
        
inference_config = Config()
inference_config.initialize_encoder_config(image_embedding_dim=IMG_EMB_DIM, 
                                          preprocessing_fn=PREPROCESSING_FN, 
                                          backbone_fn=BB_FN, 
                                          image_shape=IMG_SHAPE, 
                                          do_permute=IMG_EMB_DIM[1]<IMG_EMB_DIM[0])
inference_config.initialize_decoder_config(vocab_len=VOCAB_LEN, 
                                          attn_emb_depth=ATTN_EMB_DIM, 
                                          img_emb_dim=IMG_EMB_DIM[0], 
                                          n_rnn_units=N_RNN_UNITS)

print(f"\nINFERENCE ENCODER CONFIG:\n\t--> {inference_config.encoder_config}\n")
print(f"INFERENCE DECODER CONFIG:\n\t--> {inference_config.decoder_config}\n")

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.5 HOW TPU IMPACTS MODELS, METRICS, AND OPTIMIZERS</h3>

In order to use TPU, or [**tensorflow distribute strategy**](https://www.tensorflow.org/api_docs/python/tf/distribute) in general, certain objects will have to be created inside the **strategy's scope**

---

Here is the rule of thumb:

---

* Anything that creates variables that will be used in a distributed way must be created inside **`strategy.scope()`**.
* This includes, but is not limited to:
  - model creation
  - optimizer
  - metrics
  - sometimes, checkpoint restore
  - any custom code that creates distributed variables
* Once a variable is created inside a strategy's scope, it captures the strategy's information, and **you can use it outside the strategy's scope.**
* Unless using a high level API like **`model.fit()`**, defining something within the strategy's scope **WILL NOT automatically distribute the computation**. This will be discussed more in the section on training further down.

---

Inside the scope, everything is defined in the same way it would be outside the distribution strategy. There is, however, a particularity about the loss function which we will discuss further down as well.

**In the next cell, we instantiate the learning rate function, the loss object, and the model(s) inside the scope**

<div class="alert alert-block alert-info" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 16px;">📖 &nbsp; REFERENCE:</b><br><br>
    - <a href="https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/TPUStrategy#scope"><b>TPUStrategy - Scope</b></a><br>
    - <a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/custom_training.ipynb#scrollTo=s_suB7CZNw5W"><b>Tutorial - Custom Training With TPUs</b></a><br>
</div>

In [ ]:
print("\n... INFERENCE PREPERATION STARTING ...\n")

def prepare_for_inference(encoder_config, decoder_config, encoder_wts=None, decoder_wts=None, verbose=0):
    """ Declare required objects under TPU session scope and return ready for training
    
    Args:
        encoder_config (dict): Keyword arguments mapped to desired values for encoder model instantiation
        decoder_config (dict): Keyword arguments mapped to desired values for decoder model instantiation    
        encoder_wts (str, optional): Path to pretrained model weights for encoder
        decoder_wts (str, optional): Path to pretrained model weights for decoder
        verbose (bool, optional): Whether or not to print model information and plot lr schedule
        
    Returns:
        encoder - TBD
        decoder - TBD
        
    """
    

    # Everything must be declared within the scope when leveraging the TPU strategy
    #     - This will still function properly if scope is set to another type of accelerator
    with strategy.scope():
        # Instantiate the encoder model 
        print("\t--> CREATING & INITIALIZING ENCODER MODEL ...")
        encoder = Encoder(**encoder_config)
        encoder.build(input_shape=[REPLICA_BATCH_SIZE, *IMG_SHAPE])
        initialization_batch = encoder(
            tf.ones((REPLICA_BATCH_SIZE,*IMG_SHAPE), dtype=TARGET_DTYPE), 
            training=False,
        )
        
        # Instantiate the decoder model
        print("\t--> CREATING & INITIALIZING DECODER MODEL...")
        decoder = Decoder(**decoder_config)
        init_h, init_c = decoder.init_hidden_state(initialization_batch[:REPLICA_BATCH_SIZE], training=False)
        pred_output, h, c = decoder(
            tf.random.uniform((REPLICA_BATCH_SIZE, 1)), 
            init_h, init_c, initialization_batch, 
            training=False,
        )
        
        # Load weights after variable initialization
        if encoder_wts is not None:
            print("\t--> LOADING ENCODER MODEL WEIGHTS ...")
            encoder.load_weights(encoder_wts)
        if decoder_wts is not None:
            print("\t--> LOADING DECODER MODEL WEIGHTS ...")
            decoder.load_weights(decoder_wts)
        
    # Show the model architectures and plot the learning rate
    if verbose:
        print("\n\n... ENCODER MODEL SUMMARY...\n")
        print(encoder.summary())

        print("\n\n... DECODER MODEL SUMMARY...\n")
        print(decoder.summary())

    return encoder, decoder
    
    
print("\n... GENERATING THE FOLLOWING:")
# Instantiate our required training components in the correct scope
encoder, decoder = prepare_for_inference(
    encoder_config=inference_config.encoder_config,
    decoder_config=inference_config.decoder_config,
    encoder_wts=(ENCODER_CKPT_PATH if ENCODER_CKPT_PATH!="" else None),
    decoder_wts=(DECODER_CKPT_PATH if DECODER_CKPT_PATH!="" else None),
)

print("\n... INFERENCE PREPERATION FINISHED ...\n")

<br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="model_training">6&nbsp;&nbsp;MODEL INFERENCE&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

In this section we will define the inference routine/step as well as the final inference loop that will execute everything we have worked on up until this point.

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.1 INDIVIDUAL INFERENCE STEP</h3>

---

We use this function to represent the calculation occuring on one replica for one batch of images/labels. As this is for inference we want to pass back the predicted sequence.

***WIP***

In [ ]:
def beam_test_step(_image_batch, b_width=8, beta_factor=5):
    """ TBD 
    
    Args:
        _image_batch (Image Tensor): Batch of images
        _b_width (int): The width of the beam to use
        
    Returns:
        The most likely sequence (argmax over final k beams)
    """
    
    ##################################################################################
    # STEP 1: Get image embedding via the Encoder
    #     ----- output shape -----> (REPLICA_BATCH_SIZE, IMG_EMB_DIM)
    ##################################################################################
    image_batch_embedding = encoder(_image_batch, training=False)
    ##################################################################################

    ##################################################################################
    # STEP 2: Initialize the hidden and memory objects for decoding
    #     ----- output shapes are both ----->  (REPLICA_BATCH_SIZE, BEAM_WIDTH, N_RNN_UNITS)
    ##################################################################################
    hidden_batch, memory_batch = decoder.init_hidden_state(image_batch_embedding, training=False)
    hidden_batch = tf.repeat(tf.expand_dims(hidden_batch, axis=1), repeats=b_width, axis=1)
    memory_batch = tf.repeat(tf.expand_dims(memory_batch, axis=1), repeats=b_width, axis=1)
    ##################################################################################

    ##################################################################################
    # STEP 3: Initialize starting probabilities and sequence tokens. 
    #         These will get overwritten or appended to respectively.
    #     ----- output shapes are both ----->  (REPLICA_BATCH_SIZE, BEAM_WIDTH, 1)
    ##################################################################################
    top_probabilities_batch = tf.concat([
        tf.ones((REPLICA_BATCH_SIZE, 1, 1), dtype=tf.float32)*45,
        tf.zeros((REPLICA_BATCH_SIZE, b_width-1, 1), dtype=tf.float32),
    ], axis=-2)
    top_k_prediction_seq_batch = tf.concat([
        tf.ones((REPLICA_BATCH_SIZE, 1, 1), dtype=tf.uint8),
        tf.zeros((REPLICA_BATCH_SIZE, b_width-1, 1), dtype=tf.uint8),
    ], axis=-2)
    ##################################################################################

    ##################################################################################
    # STEP 4: Loop the token level prediction <MAX_LEN> number of times
    ##################################################################################
    for c_idx in tqdm(range(1, MAX_LEN), total=MAX_LEN-1): #MAX_LEN
        
        ##################################################################################
        # STEP 5: Initialize a list for every beam-level item we want to accumulate
        #      ----- items are -----> probabilties, predictions (indices), hidden-states, memory-states
        ##################################################################################

        probs_batch_k_list = [] # cumulative
        preds_batch_k_list = [] # cumulative

        # LSTM understanding of the previous tokens/stuff
        hidden_batch_k_list = []
        memory_batch_k_list = []

        ##################################################################################
        # STEP 6: Loop the beam level predictions <b_width> number of times
        ##################################################################################
        for k in range(b_width):     
            
            ##################################################################################
            # STEP 6: Get the kth value for probabilities, token, hidden-state, and memory-state
            #      ----- decoder_probabilities_batch_k shape -----> (REPLICA_BATCH_SIZE, 1)
            #      ----- decoder_input_batch_k shape -----> (REPLICA_BATCH_SIZE, c_idx-1) -- it grows as we go
            #      ----- hidden_batch_k shape -----> (REPLICA_BATCH_SIZE, N_RNN_UNITS)
            #      ----- memory_batch_k shape -----> (REPLICA_BATCH_SIZE, N_RNN_UNITS)
            ##################################################################################
            decoder_probabilities_batch_k = top_probabilities_batch[:, k]
            decoder_input_batch_k = top_k_prediction_seq_batch[:, k, -1:]
            hidden_batch_k = hidden_batch[:, k]
            memory_batch_k = memory_batch[:, k]

            ##################################################################################
            # STEP 7: Perform inference with the kth values captured previously and get outputs
            #      ----- prediction_batch_k shape -----> (REPLICA_BATCH_SIZE, VOCAB_LEN)
            #      ----- output_hidden_batch_k shape -----> (REPLICA_BATCH_SIZE, N_RNN_UNITS)
            #      ----- output_memory_batch_k shape -----> (REPLICA_BATCH_SIZE, N_RNN_UNITS)
            ##################################################################################
            prediction_batch_k, output_hidden_batch_k, output_memory_batch_k = \
                decoder(decoder_input_batch_k, hidden_batch_k, memory_batch_k, image_batch_embedding, training=False)

            ##################################################################################
            # STEP 8: Get the top_k (k=BEAM_WIDTH) probabilties and their respective indices (tokens)
            #      ----- raw_probs shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH)
            #      ----- raw_indices shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH)
            ##################################################################################
            raw_probs, raw_indices = tf.math.top_k(prediction_batch_k, k=b_width)
            norm_probs = tf.nn.log_softmax(raw_probs/beta_factor) # beta factor determined off-cuff
            
            ##################################################################################
            # STEP 9: Use the indices captured above and the previous predictions to generate 
            #         the top k (BEAM_WIDTH) sequences for this beam.
            #      ----- top_preds shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH, 1)
            ##################################################################################
            prep_k_seq_batch = tf.repeat(tf.expand_dims(top_k_prediction_seq_batch[:, k], axis=1), repeats=b_width, axis=1)
            prep_raw_indices_batch = tf.expand_dims(tf.cast(raw_indices, tf.uint8), axis=-1)
            prep_raw_indices_batch = tf.expand_dims(tf.where(tf.repeat(decoder_input_batch_k, repeats=b_width, axis=1)!=2, prep_raw_indices_batch[..., 0], 2), axis=-1)
            top_preds = tf.concat([
                prep_k_seq_batch,
                prep_raw_indices_batch
            ], axis=-1)

            ##################################################################################
            # STEP 10: Capture relevant values within the previously mentioned lists 
            #          as we will be using these lists as inputs into tf.concat
            #      ----- NOTE ON SHAPES -----> hidden and memory batches have a dimension added and tripled
            #                                  so that we can concat along it in the future
            ##################################################################################
            preds_batch_k_list.append(top_preds)
            probs_batch_k_list.append(norm_probs+decoder_probabilities_batch_k)
            hidden_batch_k_list.append(tf.repeat(tf.expand_dims(output_hidden_batch_k, axis=1), repeats=b_width, axis=1))
            memory_batch_k_list.append(tf.repeat(tf.expand_dims(output_memory_batch_k, axis=1), repeats=b_width, axis=1))

            
        ##################################################################################
        # STEP 11: After the beam looping is completed we concat the data along the 'beam-axis' (axis=1)
        #      ----- preds_batch_k_arr shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH*BEAM_WIDTH, c_idx-1)
        #      ----- probs_batch_k_arr shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH*BEAM_WIDTH, 1)
        #      ----- hidden_batch_k_arr shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH*BEAM_WIDTH, 1)
        #      ----- memory_batch_k_arr shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH*BEAM_WIDTH, 1)
        ##################################################################################
        preds_batch_k_arr = tf.concat(preds_batch_k_list, axis=1)    
        probs_batch_k_arr = tf.concat(probs_batch_k_list, axis=1)    
        hidden_batch_k_arr = tf.concat(hidden_batch_k_list, axis=1)    
        memory_batch_k_arr = tf.concat(memory_batch_k_list, axis=1)

        ##################################################################################
        # STEP 12: Get the top k probabilities indices and use this to capture the top k items
        #          for each arraay
        #      ----- top_k_prediction_seq_batch shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH, c_idx-1)
        #      ----- top_probabilities_batch shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH, 1)
        #      ----- hidden_batch shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH, 1)
        #      ----- memory_batch shape -----> (REPLICA_BATCH_SIZE, BEAM_WIDTH, 1)
        ##################################################################################
        filter_indices = tf.math.top_k(probs_batch_k_arr, k=b_width).indices

        top_k_prediction_seq_batch = tf.gather(preds_batch_k_arr, filter_indices, batch_dims=1)
        top_probabilities_batch = tf.expand_dims(tf.gather(probs_batch_k_arr, filter_indices, batch_dims=1), axis=-1)
        hidden_batch = tf.gather(hidden_batch_k_arr, filter_indices, batch_dims=1)
        memory_batch = tf.gather(memory_batch_k_arr, filter_indices, batch_dims=1)
        
    # Take the top most likely sequence and convert it into the appropriate datatype
    return tf.cast(tf.gather(top_k_prediction_seq_batch, tf.argmax(top_probabilities_batch, axis=1), batch_dims=1)[:, 0, :], tf.int32)

@tf.function
def distributed_beam_test_step(_img_batch, _img_ids):
    per_replica_seqs = strategy.run(beam_test_step, args=(_img_batch,))
    predictions = strategy.gather(per_replica_seqs, axis=0)
    pred_ids = strategy.gather(_img_ids, axis=0)
    return predictions, pred_ids

In [ ]:
def test_step(_image_batch):
    """ Forward pass (calculate gradients)
    
    Args:
        image_batch (): TBD
        inchi_batch (): TBD
    
    Returns:
        tbd
    """
    
    # image_batch_embedding has shape --> (REPLICA_BATCH_SIZE, IMG_EMB_DIM)
    image_batch_embedding = encoder(_image_batch, training=False)

    # hidden and memory both have the shape --> (REPLICA_BATCH_SIZE, N_RNN_UNITS)
    hidden_batch, memory_batch = decoder.init_hidden_state(image_batch_embedding, training=False)

    # decoder_input and predictions_seq share the shape --> (REPLICA_BATCH_SIZE, 1)
    decoder_input_batch = tf.ones((REPLICA_BATCH_SIZE, 1), dtype=tf.uint8)
    predictions_seq_batch = tf.ones((REPLICA_BATCH_SIZE, 1), dtype=tf.uint8)
    confidence_over_batch = tf.zeros((REPLICA_BATCH_SIZE, 1), dtype=tf.float32)

    # Teacher forcing - feeding the target as the next input
    for c_idx in range(1, MAX_LEN):
        
        # passing enc_output to the decoder
        prediction_batch, hidden_batch, memory_batch = \
            decoder(decoder_input_batch, hidden_batch, memory_batch, image_batch_embedding, training=False)
        
        confidence_over_batch += tf.reduce_max(prediction_batch, axis=-1, keepdims=True)
        
        # no teacher forcing, predicted char is next LSTMCell input
        decoder_input_batch = tf.cast(tf.expand_dims(tf.math.argmax(prediction_batch, axis=1, output_type=tf.int32), axis=1), tf.uint8)
        
        # Build the prediction sequence
        predictions_seq_batch = tf.concat([predictions_seq_batch, decoder_input_batch], axis=1)
    
    return predictions_seq_batch, confidence_over_batch

    
@tf.function
def distributed_test_step(_img_batch, _img_ids):
    per_replica_seqs, per_replica_confs = strategy.run(test_step, args=(_img_batch,))
    predictions = strategy.gather(per_replica_seqs, axis=0)
    confidences = strategy.gather(per_replica_confs, axis=0)
    pred_ids = strategy.gather(_img_ids, axis=0)
    return predictions, confidences, pred_ids

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.2 RAW INFERENCE LOOP</h3>

---

***WIP***

In [ ]:
DO_BEAM = False

# To Store The Preds
all_pred_arr = tf.zeros((1, MAX_LEN), dtype=tf.uint8)
all_pred_ids = tf.zeros((1, 1), dtype=tf.string)

if DO_BEAM:
    # Create an iterator
    dist_test_ds = iter(strategy.experimental_distribute_dataset(test_ds))
    for i in tqdm(range(TEST_STEPS), total=TEST_STEPS): 
        img_batch, id_batch = next(dist_test_ds)
        preds, pred_ids = distributed_beam_test_step(img_batch, id_batch)
        all_pred_arr = tf.concat([all_pred_arr, tf.cast(preds, tf.uint8)], axis=0)
        all_pred_ids = tf.concat([all_pred_ids, tf.expand_dims(pred_ids, axis=-1)], axis=0)
else:
    # Create an iterator
    dist_test_ds = iter(strategy.experimental_distribute_dataset(test_ds))
    for i in tqdm(range(TEST_STEPS), total=TEST_STEPS): 
        img_batch, id_batch = next(dist_test_ds)
        preds, _, pred_ids = distributed_test_step(img_batch, id_batch)
        all_pred_arr = tf.concat([all_pred_arr, tf.cast(preds, tf.uint8)], axis=0)
        all_pred_ids = tf.concat([all_pred_ids, tf.expand_dims(pred_ids, axis=-1)], axis=0)

<h3 style="font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.3 POST PROCESSING</h3>

---

***WIP***

In [ ]:
def arr_2_inchi(arr):
    inchi_str = ''
    for i in arr:
        c = int_2_tok.get(i)
        if c=="<END>":
            break
        inchi_str += c
    return inchi_str

pred_df = pd.DataFrame({
    "image_id":[x[0].decode() for x in tqdm(all_pred_ids[1:-REQUIRED_DATASET_PAD].numpy(), total=N_TEST)], 
    "InChI":[arr_2_inchi(pred_arr) for pred_arr in tqdm(all_pred_arr[1:-REQUIRED_DATASET_PAD].numpy(), total=N_TEST)]
})

pred_df = pred_df.sort_values(by="image_id").reset_index(drop=True)
pred_df.to_csv("submission.csv", index=False)